In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from matplotlib import style
import datetime as dt1
from datetime import datetime as dt

In [ ]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))